In [1]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils import seeding

from sys_data import System_Data

from julia import Main as jl

import numpy as np

import random


In [2]:
class pds_env(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self, solver="CPLEX", min_disturbance=1, max_disturbance=1):
        #---------设置求解器----------------------
        if solver == "CPLEX":
            self.solver = "CPLEX"
            
        elif solver == "Gurobi":
            self.solver = "Gurobi"
            
        else:
            raise Exception("Solver not supported!")
        
        #---------设置扰动----------------------
        self.min_disturbance = min_disturbance
        self.max_disturbance = max_disturbance
        
        # ---------读取系统数据----------------------
        self.system_data = System_Data(file_name='Case_33BW_Data.xlsx')
        varcon_lower_limit = self.system_data.Qsvc_lower_limit
        varcon_upper_limit = self.system_data.Qsvc_upper_limit
        
        # ---------初始化优化问题----------------------
        jl.eval("using " + self.solver)
        jl.include("system.jl")
        jl.init_opf_core(args_expert=self.system_data.args_expert,
                         args_step=self.system_data.args_step,
                         solver=self.solver)
        
        # ---动作状态空间相关参数设置---
        self.exploration_total = self.system_data.N_TL
        self.exploration_seq_idx = [i for i in range(self.exploration_total + 1)]
        self.action_space = spaces.Dict({
            "tieline": spaces.Discrete(self.system_data.N_TL + 1), # 0~N_TL数字,0代表什么都不做
            "varcon": spaces.Box(low=np.array(varcon_lower_limit), high=np.array(varcon_upper_limit), dtype=np.float32)
            })
        
        # self.observation_space = spaces.MultiBinary(self.system_data.N_Branch) # 如果只用tieline
        self.observation_space = spaces.Dict({
            "X_branch": spaces.MultiBinary(self.system_data.N_Branch),
            "X_load": spaces.MultiBinary(self.system_data.N_Bus-1),
            "PF": spaces.Box(low=self.system_data.S_lower_limit, high=self.system_data.S_upper_limit, dtype=np.float32),
            "QF": spaces.Box(low=self.system_data.S_lower_limit, high=self.system_data.S_upper_limit, dtype=np.float32)
            })
        
        pass
    
    def reset(self, disturbance:list=None):
        
        # initialize a list to store the load status during an episode
        self.load_value_current = 0
        self.load_value_episode = []
        
        # index to determine the instants
        self.exploration_index = 0
        
        # ================== generate disturbance ====================
        # 随机产生一个扰动破坏普通线路，在接下来的时刻中
        # generate random disturbance if no specific one is given
        
        if disturbance == None:
            temp_disturbance_set = self.system_data.disturbance_set.copy()
            # generate disturbance upper bound for this episoid   N-k的k
            num_disturbance = random.randint(self.min_disturbance, self.max_disturbance)
            # record generated disturbance
            self.disturbance = []
            # 不放回随机抽取k个线路中断
            for _ in range(num_disturbance):
                # generate one line outage at a time
                random_disturbance = random.choice(temp_disturbance_set)
                # record
                self.disturbance.append(random_disturbance)
                # remove from the set
                temp_disturbance_set.remove(random_disturbance)
                
        else:
            self.disturbance = disturbance
            
        # =============== initialize the line and var control for optimization ===============
        a = np.ones((self.system_data.N_NL, self.system_data.NT)) # 设置普通线路的灾害状态
        for dmg in self.disturbance:
            a[dmg-1, :] = 0
        
        X_tieline0 = np.zeros(self.system_data.N_TL) # tieline默认一开始都是打开的
        Q_svc0 = np.zeros(self.system_data.N_DG-1) # svc默认输出均为0
        
        jl.set_dmg(a) # 对模型设置普通线路的灾害状态
        jl.set_ResetModel(X_tieline_input=X_tieline0, Q_svc_input=Q_svc0)
        _b, self._x_load, _PF, _QF, self.load_value_current = jl.solve_ResetModel() # 需要用self记录上一步的负荷拾取情况、总负荷恢复量
        self.load_value_episode.append(self.load_value_current) # 添加到该episode的得分记录表中
        
        obs = {"X_branch": np.round(_b).astype(int),
               "X_load": np.round(self._x_load).astype(int),
               "PF": _PF,
               "QF": _QF
               }
        
        return obs     

In [4]:
env = pds_env(solver="Gurobi",min_disturbance=1, max_disturbance=5)
obs = env.reset(disturbance=None)

c:\Users\gemin\miniconda3\envs\pytorch\lib\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
